# Setup

In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import random
import datetime
import importlib
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

# Matrixlib
import matrixkit.block as blk
import matrixkit.preconditioning as prec

# Modellib
import modellib.train
import modellib.evaluate as eval
import modellib.io as io

from numpy.linalg import cond
from typing import List, Tuple

# Load test matrices and true block starts

In [4]:
# Reading original test matrices 
test_matrices = np.load("data/matrices/test_matrices_64x64_200.npy")
print(f"Loaded Matrices Shape: {test_matrices.shape}")

# Reading the test dataset (bands and labels)
_, true_blocks = io.read_from_hdf5("data/datasets/test_dataset_64_200.h5")
print(f"Test labels shape: {true_blocks.shape}")

Loaded Matrices Shape: (200, 64, 64)
Test labels shape: (200, 64)


# Identify Block Starts with Supervariable Blocking Alorithm


In [9]:
# Run algorithm for varying block sizes
svb_blocks_max_4 = blk.supervariable_blocking(test_matrices, 4)
svb_blocks_max_8 = blk.supervariable_blocking(test_matrices, 8)
svb_blocks_max_16 = blk.supervariable_blocking(test_matrices, 16)
svb_blocks_max_32 = blk.supervariable_blocking(test_matrices, 32)

# Verify Shapes 
print(f"SVB Block Starts Shape: {svb_blocks_max_8.shape}")

SVB Block Starts Shape: (200, 64)


# Evaluate results

In [10]:
# Get performance metrics for each blocksize limit
svb_metrics_dict = {}
svb_metrics_dict['max_4'] = eval.calculate_metrics(true_blocks, svb_blocks_max_4)
svb_metrics_dict['max_8'] = eval.calculate_metrics(true_blocks, svb_blocks_max_8)
svb_metrics_dict['max_16'] = eval.calculate_metrics(true_blocks, svb_blocks_max_16)
svb_metrics_dict['max_32'] = eval.calculate_metrics(true_blocks, svb_blocks_max_32)

Element-wise Accuracy: 0.6965625
Classification Report:
              precision    recall  f1-score   support

    no block       0.90      0.75      0.82     11516
       block       0.09      0.23      0.13      1284

    accuracy                           0.70     12800
   macro avg       0.50      0.49      0.47     12800
weighted avg       0.82      0.70      0.75     12800

Confusion Matrix:
[[8616 2900]
 [ 984  300]]
True Positives: 300
False Negatives: 984
True Negatives: 8616
False Positives: 2900
Element-wise Accuracy: 0.7971875
Classification Report:
              precision    recall  f1-score   support

    no block       0.90      0.87      0.89     11516
       block       0.09      0.11      0.10      1284

    accuracy                           0.80     12800
   macro avg       0.49      0.49      0.49     12800
weighted avg       0.82      0.80      0.81     12800

Confusion Matrix:
[[10060  1456]
 [ 1140   144]]
True Positives: 144
False Negatives: 1140
True Negatives

# Save computed Block Starts from SVB

In [11]:
# Save blockstarts 
np.save('data/blockstarts/svb_predictions_max_4.npy', svb_blocks_max_4)
np.save('data/blockstarts/svb_predictions_max_8.npy', svb_blocks_max_8)
np.save('data/blockstarts/svb_predictions_max_16.npy', svb_blocks_max_16)
np.save('data/blockstarts/svb_predictions_max_32.npy', svb_blocks_max_32)